In [1]:
!pip install -r requirements.txt
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.23.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.2 kB)
ERROR: Ignored the following versions that require a different python version: 2.1.0 Requires-Python >=3.10; 2.1.0rc1 Requires-Python >=3.10; 2.1.1 Requires-Python >=3.10
ERROR: Could not find a version that satisfies the requirement tensorflow==2.10.0 (from versions: 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.18.0rc0)
ERROR: No matching distribution found for tensorflow==2.10.0
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install albumentations


Defaulting to user installation because normal site-packages is not writeable


In [3]:
!python3 -m pip install --upgrade pip
!pip install ImageLoader
!pip install tqdm


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached imageloader-0.0.5.tar.gz (58 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [108 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'imageloader' extension
      creating build
      creating build/temp.macosx-10.9-universal2-3.9
      clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -iwithsysroot/System/Library/Frameworks/System.framework/PrivateHeaders -iwithsysroot/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/Headers -arch arm64 -arch x86_64 -Werror=implicit-function-declaration -Wno-error=unreachable-code -I/Library/Developer/CommandLineToo

In [4]:
# Importing the necessary libraries

import tensorflow as tf
import albumentations as albu
import numpy as np
import gc
import pickle
import matplotlib.pyplot as plt
from keras.callbacks import CSVLogger
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, precision_score, recall_score, accuracy_score, f1_score
from ModelArchitecture.DiceLoss import dice_metric_loss
from ModelArchitecture import DUCK_Net
from ImageLoader.ImageLoader2D import load_data

/Users/majjarisuman/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/majjarisuman/Library/Python/3.9/lib/python/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.17 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
# Checking the number of GPUs available

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [6]:
# Setting the model parameters
img_size = 256
dataset_type = 'Kvasir-SEG'  # Options: kvasir/cvc-clinicdb/cvc-colondb/etis-laribpolypdb
learning_rate = 1e-4
seed_value = 58800
filters = 17  # Number of filters, the paper presents the results with 17 and 34
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

# Get the current timestamp
ct = datetime.now()

# Model type
model_type = "DuckNet"

# Paths for saving progress, plots, and model
progress_path = f'ProgressFull/{dataset_type}_progress_csv_{model_type}_filters_{filters}_{ct}.csv'
progressfull_path = f'ProgressFull/{dataset_type}_progress_{model_type}_filters_{filters}_{ct}.txt'
plot_path = f'ProgressFull/{dataset_type}_progress_plot_{model_type}_filters_{filters}_{ct}.png'
model_path = f'ModelSaveTensorFlow/{dataset_type}/{model_type}_filters_{filters}_{ct}'



# Training settings
EPOCHS = 1
min_loss_for_saving = 0.2


In [7]:
# Loading the data
X, Y = load_data(img_size, img_size, -1, 'Datasets/Kvasir-SEG')

Total number of images found: 800
Resizing training images and masks: 800


800it [00:03, 228.02it/s]


In [8]:
# Splitting the data, using seed for reproducibility
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, shuffle=True, random_state=seed_value)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.111, shuffle=True, random_state=seed_value)


In [9]:
# Defining the augmentations

aug_train = albu.Compose([
    albu.HorizontalFlip(),
    albu.VerticalFlip(),
    albu.ColorJitter(brightness=(0.6,1.6), contrast=0.2, saturation=0.1, hue=0.01, always_apply=True),
    albu.Affine(scale=(0.5,1.5), translate_percent=(-0.125,0.125), rotate=(-180,180), shear=(-22.5,22), always_apply=True),
])

def augment_images():
    x_train_out = []
    y_train_out = []

    for i in range (len(x_train)):
        ug = aug_train(image=x_train[i], mask=y_train[i])
        x_train_out.append(ug['image'])  
        y_train_out.append(ug['mask'])

    return np.array(x_train_out), np.array(y_train_out)

In [10]:
# Creating the model

model = DUCK_Net.create_model(img_height=img_size, img_width=img_size, input_chanels=3, out_classes=1, starting_filters=filters)

Starting DUCK-Net


In [11]:
# Compiling the model

model.compile(optimizer=optimizer, loss=dice_metric_loss)

In [12]:
pip install --upgrade tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
# # Training the model

# step = 0

# for epoch in range(0, EPOCHS):
    
#     print(f'Training, epoch {epoch}')
#     print('Learning Rate: ' + str(learning_rate))

#     step += 1
        
#     image_augmented, mask_augmented = augment_images()
    
#     csv_logger = CSVLogger(progress_path, append=True, separator=';')
    
#     model.fit(x=image_augmented, y=mask_augmented, epochs=1, batch_size=4, validation_data=(x_valid, y_valid), verbose=1, callbacks=[csv_logger])
    
#     prediction_valid = model.predict(x_valid, verbose=0)
#     loss_valid = dice_metric_loss(y_valid, prediction_valid)
    
#     loss_valid = loss_valid.numpy()
#     print("Loss Validation: " + str(loss_valid))
        
#     prediction_test = model.predict(x_test, verbose=0)
#     loss_test = dice_metric_loss(y_test, prediction_test)
#     loss_test = loss_test.numpy()
#     print("Loss Test: " + str(loss_test))
        
#     with open(progressfull_path, 'a') as f:
#         f.write('epoch: ' + str(epoch) + '\nval_loss: ' + str(loss_valid) + '\ntest_loss: ' + str(loss_test) + '\n\n\n')
    
#     if min_loss_for_saving > loss_valid:
#         min_loss_for_saving = loss_valid
#         print("Saved model with val_loss: ", loss_valid)
#         model.save(model_path)
        
#     del image_augmented
#     del mask_augmented

#     gc.collect()

import os
import gc
import tensorflow as tf

step = 0
progress_dir = 'ProgressFull'
if not os.path.exists(progress_dir):
    os.makedirs(progress_dir)

# if not os.path.exists(dataset_type):
#     os.makedirs(dataset_type)

for epoch in range(0, EPOCHS):
    
    print(f'Training, epoch {epoch}')
    print('Learning Rate: ' + str(learning_rate))

    step += 1
        
    image_augmented, mask_augmented = augment_images()
    
    csv_logger = CSVLogger(os.path.join(progress_dir, 'progress.csv'), append=True, separator=';')
    
    model.fit(x=image_augmented, y=mask_augmented, epochs=1, batch_size=4, validation_data=(x_valid, y_valid), verbose=1, callbacks=[csv_logger])
    
    prediction_valid = model.predict(x_valid, verbose=0)
    loss_valid = dice_metric_loss(y_valid, prediction_valid)
    
    loss_valid = loss_valid.numpy()
    print("Loss Validation: " + str(loss_valid))
        
    prediction_test = model.predict(x_test, verbose=0)
    loss_test = dice_metric_loss(y_test, prediction_test)
    loss_test = loss_test.numpy()
    print("Loss Test: " + str(loss_test))
        
    with open(progressfull_path, 'a') as f:
        f.write('epoch: ' + str(epoch) + '\nval_loss: ' + str(loss_valid) + '\ntest_loss: ' + str(loss_test) + '\n\n\n')
    
    # if min_loss_for_saving > loss_valid:
    #     min_loss_for_saving = loss_valid
    #     print("Saved model with val_loss: ", loss_valid)
    model.save(f'{model_path}.h5', save_format='h5')
        
    del image_augmented
    del mask_augmented

    gc.collect()


Training, epoch 0
Learning Rate: 0.0001
 14/160 ━━━━━━━━━━━━━━━━━━━━ 14:36 6s/step - loss: 0.7513

KeyboardInterrupt: 

In [71]:
pip install keras==2.11.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-2.11.0-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires keras>=3.2.0, but you have keras 2.11.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [72]:
import tensorflow as tf
import os


print("Model Path : ", model_path)

model = tf.keras.models.load_model(model_path, custom_objects={'dice_metric_loss': dice_metric_loss})

prediction_train = model.predict(x_train, batch_size=4)
prediction_valid = model.predict(x_valid, batch_size=4)
prediction_test = model.predict(x_test, batch_size=4)

print("Predictions done")

dice_train = f1_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))
dice_test = f1_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
dice_valid = f1_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))

print("Dice finished")


miou_train = jaccard_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                           np.ndarray.flatten(prediction_train > 0.5))
miou_test = jaccard_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
miou_valid = jaccard_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                           np.ndarray.flatten(prediction_valid > 0.5))

print("Miou finished")


precision_train = precision_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                  np.ndarray.flatten(prediction_train > 0.5))
precision_test = precision_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                                 np.ndarray.flatten(prediction_test > 0.5))
precision_valid = precision_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                  np.ndarray.flatten(prediction_valid > 0.5))

print("Precision finished")


recall_train = recall_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                            np.ndarray.flatten(prediction_train > 0.5))
recall_test = recall_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                           np.ndarray.flatten(prediction_test > 0.5))
recall_valid = recall_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                            np.ndarray.flatten(prediction_valid > 0.5))

print("Recall finished")


accuracy_train = accuracy_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                np.ndarray.flatten(prediction_train > 0.5))
accuracy_test = accuracy_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                               np.ndarray.flatten(prediction_test > 0.5))
accuracy_valid = accuracy_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                np.ndarray.flatten(prediction_valid > 0.5))


print("Accuracy finished")


final_file = 'results_' + model_type + '_' + str(filters) + '_' + dataset_type + '.txt'
print(final_file)

with open(final_file, 'a') as f:
    f.write(dataset_type + '\n\n')
    f.write('dice_train: ' + str(dice_train) + ' dice_valid: ' + str(dice_valid) + ' dice_test: ' + str(dice_test) + '\n\n')
    f.write('miou_train: ' + str(miou_train) + ' miou_valid: ' + str(miou_valid) + ' miou_test: ' + str(miou_test) + '\n\n')
    f.write('precision_train: ' + str(precision_train) + ' precision_valid: ' + str(precision_valid) + ' precision_test: ' + str(precision_test) + '\n\n')
    f.write('recall_train: ' + str(recall_train) + ' recall_valid: ' + str(recall_valid) + ' recall_test: ' + str(recall_test) + '\n\n')
    f.write('accuracy_train: ' + str(accuracy_train) + ' accuracy_valid: ' + str(accuracy_valid) + ' accuracy_test: ' + str(accuracy_test) + '\n\n\n\n')

print('File done')


Model Path :  ModelSaveTensorFlow/Kvasir-SEG/DuckNet_filters_17_2024-09-28 14:36:34.563555


ValueError: File format not supported: filepath=ModelSaveTensorFlow/Kvasir-SEG/DuckNet_filters_17_2024-09-28 14:36:34.563555. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(ModelSaveTensorFlow/Kvasir-SEG/DuckNet_filters_17_2024-09-28 14:36:34.563555, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).